<a href="https://colab.research.google.com/github/sielerod/search_stackoverflow/blob/master/Read_Stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Objetivo:**   
* Capturar as perguntas mais frequentes sobre Python no stackoverflow
* Armazenar para cada pergunta: link, breve descrição da pergunta, quantidade de votos e visualizações, pergunta, respostas com melhor avaliação


**Fonte:** https://stackoverflow.com/questions/


In [103]:
import requests # Getting Webpage content
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup as bs # Scraping webpages
from time import sleep


#Leitura do dado cru no Stackoverflow
**read_stackoverflow_overview(tags=[], tab='Frequent', selector='question-summary', pages)**

Leitura do resumo das perguntas mais frequentes no stackoverflow com base em alguns parâmetros de busca. 

Retorna um objeto requests contendo o resultado de requests.get

* tags: argumento opcional com lista  de strings contendo os tipos de pergunta para seleação. Ex.: ['python', 'php', 'javascript']
>ex. de URL para página com mais de 1 tag: https://stackoverflow.com/questions/tagged/sql+sql-server?tab=Frequent

* tab: string com tipo de ordenação a ser aplicado, pode ser:
'Frequent' (opção default), 'Votes', 'Unanswered', 'Bounties', 'Active', 'Newest'

* Selector: seleção dos trechos do html a serem retornados. Por default, será question-summary

* pages: número de páginas para leitura



In [122]:
def read_stackoverflow_overview(tags=[], tab='Frequent', selector='question-summary', pages=5):
  link = 'https://stackoverflow.com/questions'

  if tags:
    tags_link = '/tagged/'
    pre=''
    for t in tags:
      tags_link += pre + t
      pre = '+' 
    link += tags_link

  link += '?tab='+tab

  questions_text = ''
  soup_selection = []
  for page in range(1,pages+1):
    page_link = '&page='+str(page)

    try:
      request = requests.get(link+page_link)
      request.raise_for_status()
      #questions_text += request.text
      try:
        soup = bs(request.text, 'html.parser')
        soup_selection.append(soup.select('.'+selector))
      except: print ("Could not transform to soup object by selecting ",selector)
    except HTTPError:
      print ("Could not download page ", page)

    sleep(0.05)

  return soup_selection


In [123]:
questions_overview_raw = read_stackoverflow_overview(tags=['python','django'],tab='Frequent',selector='question-summary', pages=2)
type(questions_overview_raw)

list

#Transformação do dado cru coletado do Stackoverflow em dicionário
**questions_overview(questions_overview_raw)**

O dicionário deve conter a visão geral das perguntas do stackoverflow, com:

* link
* brief_description
* votes
* views

###Análise do padrão da página HTML para captura de informações relevantes:

Em "question-summary", temos as seguintes informações relevantes:

1.   class = statscontainer, com:
*   Número de votos em class="vote-count-post "
>```<span class="vote-count-post high-scored-post"><strong>2473</strong></span>```

*   Número de respostas aceitas em class="status answered-accepted" 
>```<div class="status answered-accepted"><strong>23</strong>answers</div>```

*   Conteúdo e Title contendo quantidade de views em class="views supernova" 
>```<div class="views supernova" title="307,292 views">307k views</div>```

2.   class = summary, com:
* class="question-hyperlink" contendo em *href* parte do link para compor link de acesso à página detalhada da pergunta e Título da pergunta
>``` <a href="/questions/15112125/how-to-test-multiple-variables-against-a-value" class="question-hyperlink">How to test multiple variables against a value?</a>```

*   Breve resumo em class="excerpt"
>```<div class="excerpt"> brief description of the question ...</div>```

*   Tags em class="post-tag"
>```<a href="/questions/tagged/python" class="post-tag" title="show questions tagged 'python'" rel="tag">python</a>```





In [148]:
def questions_overview(questions_overview_raw):
  questions_overview = { "questions":[]}
  for soups in questions_overview_raw:
    for q in soups:
      q_title = q.select_one('.question-hyperlink').getText()
      q_link = 'https://stackoverflow.com'+q.select_one('.question-hyperlink').get('href')
      q_summary = q.select_one('.excerpt').getText()
      q_vote_count =  q.select_one('.vote-count-post').getText()
      #q_answered_accepted = q.select_one(".answered-accepted.mini-counts").getText()
      q_views =  q.select_one('.views').attrs['title']
      q_tags = []
      for tag in q.select('.post-tag'): q_tags.append(tag.getText())

      questions_overview['questions'].append({
          'title': q_title,
          'link': q_link,
          'summary': q_summary,
          'vote_count': q_vote_count,
          'views': q_views,
          'tags': q_tags,
      })
  
  return questions_overview

In [149]:
questions_dic = questions_overview(questions_overview_raw)
type(questions_dic)

dict

#Exemplos de como acessar a informação no dicionário:

In [158]:
print('Lista com links: ',[q['link'] for q in questions_dic['questions']])

print('\n Acesso a dados de um link específico\n--- Link: ',questions_dic['questions'][49]['link'])

print('\n--- Título: ', questions_dic['questions'][49]['title'])

print('\n--- Breve Descrição: ', questions_dic['questions'][49]['summary'])

print('\n--- Contagem de votos: ', questions_dic['questions'][49]['vote_count'])

print('\n--- Contagem de visualizações: ', questions_dic['questions'][49]['views'])

print('\n--- Lista como tags: ', questions_dic['questions'][49]['tags'])

len(questions_dic['questions'])

Lista com links:  ['https://stackoverflow.com/questions/23708898/pip-is-not-recognized-as-an-internal-or-external-command', 'https://stackoverflow.com/questions/573618/set-up-a-scheduled-job', 'https://stackoverflow.com/questions/8000022/django-template-how-to-look-up-a-dictionary-value-with-a-variable', 'https://stackoverflow.com/questions/5100539/django-csrf-check-failing-with-an-ajax-post-request', 'https://stackoverflow.com/questions/8609192/differentiate-null-true-blank-true-in-django', 'https://stackoverflow.com/questions/1156246/having-django-serve-downloadable-files', 'https://stackoverflow.com/questions/2428092/creating-a-json-response-using-django-and-python', 'https://stackoverflow.com/questions/7446187/no-module-named-pkg-resources', 'https://stackoverflow.com/questions/2642613/what-is-related-name-used-for-in-django', 'https://stackoverflow.com/questions/20306981/how-do-i-integrate-ajax-with-django-applications', 'https://stackoverflow.com/questions/629551/how-to-query-as-

100

Próximos passos:


1.   Enriquecer questions_dic com a informação detalhada da pergunta e conteúdo da resposta com melhor avaliação
2.   Limpar dados em questions_dic para remover caracteres irrelevantes, como: \n, \t, artigos, pronomes

